Complete explanation of the steps to train YOLOv3 can be found here: https://github.com/AlexeyAB/darknet  

**Preparation**:  
- Create a folder as a working area in the google drive folder, e.g. ``YOLOv3``
- Upload the ``dataset.zip`` into the ``YOLOv3`` folder
- Create a notebook in google colab and run below steps in the google colab notebook

## 1 Mounting google drive

In [ ]:
import cv2
import glob
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 2 Clone the ``Darknet`` repository and compile ``Makefile``  

In [ ]:
# clone the Darknet repository
!git clone https://github.com/AlexeyAB/darknet.git

# change directory to the darknet folder
%cd darknet

# allowing usage of OpenCV and GPU
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

# compile
!make

## 3 Edit configuration file

In [ ]:
# copy & rename the cfg file
!cp cfg/yolov3-tiny.cfg cfg/yolov3tiny_custom.cfg

# define values of classes and filters
classes = 3
filters = (classes + 5) * 3

# max_batches are expected not to less than 6000
max_batches = max(6000, classes * 2000)

# Edit classes & filters
!sed -i 's/batch=1/batch=64/' cfg/yolov3tiny_custom.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3tiny_custom.cfg
!sed -i 's/max_batches = 500200/max_batches = $max_batches/' cfg/yolov3tiny_custom.cfg
!sed -i '135 s@classes=80@classes=$classes@' cfg/yolov3tiny_custom.cfg
!sed -i '177 s@classes=80@classes=$classes@' cfg/yolov3tiny_custom.cfg
!sed -i '127 s@filters=255@filters=$filters@' cfg/yolov3tiny_custom.cfg
!sed -i '171 s@filters=255@filters=$filters@' cfg/yolov3tiny_custom.cfg

According to https://github.com/AlexeyAB/darknet, the ``steps`` variable in the configuration ``.cfg`` file can be changed to 80% and 90% of max_batches value (i.e. 4800,5400).  
Manually change the steps in the config file can be done.

## 4 Prepare the dataset

In [ ]:
# create object sub-folder inside data folder
!mkdir data/obj

# unzip the image dataset, move them to the object folder
!unzip /content/gdrive/MyDrive/YOLOv3/dataset.zip -d data/obj

## 5 Create ``data.obj``, ``data.names`` and ``train.txt``

In [ ]:
# create obj.names file containing the name of the classes
!echo -e 'Hotdog\nPizza\nHamburger' > data/obj.names

# create obj.data file containing the class, train.txt and save location information
!echo -e 'classes = $classes\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = /content/gdrive/MyDrive/YOLOv3' > data/obj.data

In [ ]:
# create train.txt file
images_list = glob.glob("data/obj/*[jpg|png|jpeg]")
print(images_list)

file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

## 6 Download the pre-trained weights  
The pre-trained weights for the YOLOv3-tiny can be downloaded from https://pjreddie.com/media/files/yolov3-tiny.weights.  
Alternatively, the pre-trained YOLOv3-tiny weights can also use the pre-trained weights provided at https://github.com/AlexeyAB/darknet that can be downloaded from here: https://drive.google.com/file/d/18v36esoXCh-PsOKwyP2GWrpYDptDY8Zf/view

In [ ]:
# this is from pjreddie
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

## 7 Start training

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3tiny_custom.cfg yolov3-tiny.weights -dont_show -clear

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 376551, rewritten_bbox = 0.000000 % 

 5903: 0.267418, 0.210023 avg loss, 0.000010 rate, 2.192683 seconds, 377792 images, 0.088540 hours left
Loaded: 0.000038 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.781068), count: 4, class_loss = 0.329415, iou_loss = 0.153972, total_loss = 0.483387 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 376555, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.822550), count: 4, class_loss = 0.317122, iou_loss = 0.109977, total_loss = 0.427099 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.

**Summary**
The average loss on the last epoch is 0.373.  
Training the YOLOv3-tiny for 3 classes for 6000 epochs takes ~ 3 hours.  

**Note**  
The weights at epoch of multiples of 1000 and last epoch are automatically saved once the training is done. Download the last weights for prediction.  
Moreover, download the ``obj.data``, ``obj.names``, ``yolov3tiny_custom.cfg`` files from the ``Darknet`` folder.